# Experiment - Summeringar utan body

Provar att ta bort metodens body för att se om CodeT5 kan generera bättre summeringar.
Pierre föreslog att vi kunde testa detta, efter vårt möte den 3 mars.
Om detta fungerar kan vi prova skicka vidare output till SBERT och utforska den lösningen.

In [43]:
!pip install transformers
!pip install sentence-transformers

In [44]:
from transformers import RobertaTokenizer, T5ForConditionalGeneration

tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-base')
code_t5 = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-base-multi-sum')

In [45]:
def summarize_code(text):
    input_ids = tokenizer(text, return_tensors="pt").input_ids
    generated_ids = code_t5.generate(input_ids)
    summary = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return summary

## Här är några summeringar av funktioner med deras implementation


In [46]:
print(summarize_code('''
def greet(name):
  print("Hello, " + name)
'''))

print(summarize_code('''
def greet(name = None):
  if name:
    print("Hello, " + name + "!")
  else:
    print("Hello, there!")
'''))

print(summarize_code('''
def svg_to_image(string, size=None):
    if isinstance(string, unicode):
        string = string.encode('utf-8')
        renderer = QtSvg.QSvgRenderer(QtCore.QByteArray(string))
    if not renderer.isValid():
        raise ValueError('Invalid SVG data.')
    if size is None:
        size = renderer.defaultSize()
        image = QtGui.QImage(size, QtGui.QImage.Format_ARGB32)
        painter = QtGui.QPainter(image)
        renderer.render(painter)
    return image
'''))

print(summarize_code('''
def greet(name, message="How are you today?"):
  print("Hello, " + name + "!")
  print(message)
'''))

print(summarize_code('''
def binary_search(arr, x):
    low = 0
    high = len(arr) - 1
    mid = 0
 
    while low <= high:
 
        mid = (high + low) // 2
 
        # If x is greater, ignore left half
        if arr[mid] < x:
            low = mid + 1
 
        # If x is smaller, ignore right half
        elif arr[mid] > x:
            high = mid - 1
 
        # means x is present at mid
        else:
            return mid
 
    # If we reach here, then the element was not present
    return -1
'''))

# introduce some bugs:
print(summarize_code('''
def binary_search(arr, x):
    low = 0
    high = len(arr) - 1
    mid = 0
 
    while low < high:
 
        mid = (high + low) // 2
 
        # If x is greater, ignore left half
        if arr[mid] < x:
            low = mid + 1
 
        # If x is smaller, ignore right half
        elif arr[mid] > x:
            high = mid - 1
 
        # means x is present at mid
        else:
            return mid
'''))

# Bug 1
print(summarize_code('''
def dist(x1, x2, y1, y2):
  return 45
'''))

# Bug 2 - ingen roten ur
print(summarize_code('''
def dist(x1, x2, y1, y2):
  return pow(x1-x2, 2) + pow(y1-y2, 2)
'''))

# Bugfix
print(summarize_code('''
def dist(x1, x2, y1, y2):
  return pow(pow(x1-x2, 2) + pow(y1-y2, 2), 0.5)
'''))

Print a greet message.
Greet the name of the node.
Convert a SVG string to a QImage.
Greet a name and a message.
Binary search for a node in a list of node objects.
Binary search for a node in a list.
Return the distance between two points.
Distance between two points.
Return the distance between two points.


## Här är motsvarande summeringar av enbart signaturerna
Anledningen till att jag inte har med returns är för att det kan finnas flera branches genom koden med olika returns, vilket gör det otydligt vad man ska ha kvar / ta bort.

In [51]:
print(summarize_code('''
def greet(name):
'''))

print(summarize_code('''
def greet(name = None):
'''))

print(summarize_code('''
def svg_to_image(string, size=None):
'''))

print(summarize_code('''
def greet(name, message="How are you today?"):
'''))

print(summarize_code('''
def binary_search(arr, x):
'''))

# introduce some bugs:
print(summarize_code('''
def binary_search(arr, x):
'''))

# Bug 1
print(summarize_code('''
def dist(x1, x2, y1, y2):
'''))

# Bug 2 - ingen roten ur
print(summarize_code('''
def dist(x1, x2, y1, y2):
'''))

# Bugfix
print(summarize_code('''
def dist(x1, x2, y1, y2):
'''))

Greedy function to create a greet.
Greedy function to call when a node is greeted.
Convert SVG string to image.
A function to print a greet message.
Calculate the distance between 2 points
Binary search for an element in an array.
Distance between two points.
Distance between two points.
Distance between two points.


De första översättningarna blev märkbart sämre vilket inte är så konstigt då mycket information om funktionen låg i implementationen. Möjligtvis hade detta kunnat bli bättre med en doc-string?

## Prova med signatur och doc-string

In [61]:
print(summarize_code('''
def binary_search(arr, x):
    """This is an incorrect docstring"""
'''))

print(summarize_code('''
def binary_search(arr, x):
    """Binary search for x in a tree"""
'''))

print(summarize_code('''
def svg_to_image(svg):
    """Binary search for x in a tree"""
'''))

print(summarize_code('''
def dist(x1, x2, y1, y2):
    """Calculate this distance between two points in 3D"""
'''))

Binary search for a non - empty list.
Binary search for x in a tree.
Binary search for x in a tree
Calculate this distance between two points in 3D.


Det verkar som att väldigt mycket vikt ges åt vad som står i doc-stringen, ibland så pass mycket att den bara kopieras (dock inte alltid). Detta är problematiskt eftersom utvecklare troligtvis ofta glömmer att uppdatera doc-strings när de ändrar i koden. Om vi dock antar att doc-strings alltid är uppdaterade hade vi nog kunnat använda de som input till SBERT och antagligen få ganska bra resultat.

## Testar även med lite fler signaturer...

In [48]:
signatures = [
    "def add(x):",
    "def add(x, y):",
    "def add(item):",
    "def add(items):",
    
    "def fit(lr=0.1):",
    "def fit(lr=3):",
    "def fit(lr=0.2, batch_size=16):"
    "def fit(lr=0.2, batch_size=16, epochs=3):",
    "def fit(examples, lr=0.2, batch_size=16, epochs=3):",
    "def fit(examples, lr, batch_size, epochs):",
    "def fit(examples):",
]
for s in signatures:
    print(summarize_code(s))

Add a value to the set.
Add a point to the map.
Add an item to the list
Add items to the list.
Fit the model to the current model.
Fit the model to the current model.
Fit a model to a model with a given learning rate.
Fit a model to a list of examples.
Fit the model with the given examples.
Fit the model to the examples.


Bra namn på parametrar ger bättre summeringar. Däremot verkar CodeT5 inte fånga upp ändringar i antalet parametrar eller default värden. Hade man kunnat träna fram detta? 

## Slutsats

CodeT5 ger inte tillräckligt bra summeringar out of the box i vårt fall.
Möjliga sätt att gå vidare:
- Undersök om det hade gått att fine-tuna CodeT5
- Prova om det funkar bättre med CodeBERT
- Antag att doc-strings i koden är uppdaterade och använd dem som input (Den lättaste lösningen)